# Analysis
In this notebook we have all the functions that are relevant to the analysis of the model. We are working with the following functions:

#### evaluate_all_graphs
##### inputs:
1) model: The GNN model we train
2) prepared_training_data: The list of the pytorch graphs we are working with for training (some of the edges are covered)
3) prepared_test_data: The list of the pytorch graphs we are working with for testing the model (the edges are not covered)
4) edges_to_hide: The list of the edge IDs to hide
5) criterion: The loss function we are working with
##### output:
The average loss of the graphs.

#### make_an_estimation
##### inputs: 
1) edges_to_hide: The list of the edge IDs to hide
2) model: The GNN model we train
3) data_training: A single graph of the list of the pytorch graphs we are working with for training (some of the edges are covered)
4) data_testing: A single graph of the the list of the pytorch graphs we are working with for testing the model (the edges are not covered)
5) criterion: The loss function we are working with
##### outputs:
1) The loss of the output of the model
2) The output of the model

In [1]:
#Necessary imports
import sumolib
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import random

import torch
import torch_geometric
import torch_geometric.data as Data
import torch_geometric.utils as pyg_utils

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import json
import sys

In [4]:
def evaluate_all_graphs(model, prepared_training_data, prepared_test_data, edges_to_hide, criterion):
    total_loss = 0
    for i in range(len(prepared_training_data)):
        output = model(prepared_training_data[i].edge_attr, prepared_training_data[i].edge_index)
        loss = criterion(output[edges_to_hide], prepared_test_data[i].edge_attr[edges_to_hide])
        total_loss += loss
    return total_loss/len(prepared_training_data)

In [3]:
def make_an_estimation(edges_to_hide, model, data_training, data_testing, criterion):
    edge_features = data_training.edge_attr
    max_value = edge_features.max()
    normalized_edge_features = edge_features / max_value
    data_training.edge_attr = normalized_edge_features
    output = model(data_training.edge_attr, data_training.edge_index)
    loss = criterion(output[edges_to_hide], data_testing.edge_attr[edges_to_hide])
    return loss, output

In [5]:
def GEH(M, C):
    geh = np.sqrt(2*np.power((float(M) - float(C)), 2) / (float(M) + float(C)))
    return geh

In [1]:
def evaluate_all_graphs_GEH(model, prepared_training_data, prepared_test_data, edges_to_hide):
    total_loss = 0
    for i in range(len(prepared_training_data)):
        output = model(prepared_training_data[i].edge_attr, prepared_training_data[i].edge_index)
        cur_loss = 0 
        for j in edges_to_hide:
            cur_loss += GEH( prepared_test_data[i].edge_attr[j], output[j])
        total_loss += cur_loss
    return total_loss/len(prepared_training_data)

In [ ]:
def evaluate_all_graphs_forecasting(model, prepared_training_data, prepared_training_data_hidden ,prepared_test_data, prepared_test_data_hidden, edges_to_hide, criterion, time_step):
    total_train_loss = 0
    total_test_loss = 0
    number_to_divide_with_train = 0
    number_to_divide_with_test = 0
    for i in range(len(prepared_training_data_hidden)):
        train_loss_in_list = 0
        for j in range(len(prepared_training_data_hidden[i])-1-time_step):
            output = model(prepared_training_data_hidden[i][j].edge_attr, prepared_training_data_hidden[i][j].edge_index)
            loss = criterion(output[edges_to_hide], prepared_training_data[i][j+time_step].edge_attr[edges_to_hide])
            train_loss_in_list += loss
            number_to_divide_with_train += 1
        total_train_loss += train_loss_in_list

    for i in range(len(prepared_test_data_hidden)):
        test_loss_in_list = 0
        for j in range(len(prepared_test_data_hidden[i])-1-time_step):
            output = model(prepared_test_data_hidden[i][j].edge_attr, prepared_test_data_hidden[i][j].edge_index)
            loss = criterion(output[edges_to_hide], prepared_test_data[i][j+time_step].edge_attr[edges_to_hide])
            test_loss_in_list += loss
            number_to_divide_with_test += 1
        total_test_loss += test_loss_in_list
    return total_train_loss/number_to_divide_with_train, total_test_loss/number_to_divide_with_test
